In [4]:
import numpy as np
import pandas as pd
import Linear_Regression as LR
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal

In [5]:
a_none = np.array(None)
data = pd.read_csv('data.csv')
data_train = data[0:500].select_dtypes(include='number')
price = data_train['price']
data_train = data_train.drop(['price', 'Unnamed: 0.1',	'Unnamed: 0'], axis=1)
data_train.insert(0, 'intersection', pd.DataFrame([1]*len(data_train)))
data_train.insert(data_train.shape[1], 'price', price)

data_test = data[500:].select_dtypes(include='number')
price = data_test['price']
data_test = data_test.drop(['price', 'Unnamed: 0.1',	'Unnamed: 0'], axis=1)
data_test.insert(0, 'intersection', pd.DataFrame([1]*len(data_test), index=list(data_test.index)))
data_test.insert(data_test.shape[1], 'price', price)
x_test = data_test.drop('price', axis=1)
y_test = data_test['price']